In [1]:
import ast
import datetime
import os
import random
import re
import warnings

import autogluon.tabular
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.linear_model
import sklearn.metrics
import sklearn.model_selection

import dynmodel

pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 200

warnings.filterwarnings("ignore", message="Can't initialize NVML")

In [2]:
DIRECTORY_DATA = 'data/model-autogluon'
os.makedirs(DIRECTORY_DATA, exist_ok=True)

# Preparing data

In [3]:
# FILENAME_DF_ALL_CLEAN = None
#FILENAME_DF_ALL_CLEAN = 'data/20241230_173555/df_all_clean.csv'
# FILENAME_DF_ALL_CLEAN = 'data/20250128_094430/df_all_clean.csv'
# FILENAME_DF_ALL_CLEAN = 'data/20250220_094622_halfway/df_all_clean.csv'
# FILENAME_DF_ALL_CLEAN = 'data/20250220_094622/df_all_clean.csv'
# FILENAME_DF_ALL_CLEAN = 'data/20250223_150717_halfway/df_all_clean.csv'
# FILENAME_DF_ALL_CLEAN = 'data/20250223_150717/df_all_clean.csv'
FILENAME_DF_ALL_CLEAN = 'data/20250228_090518/df_all_clean.csv'

FILENAME_DF_ALL_CLEAN_VEHICLES = None
# FILENAME_DF_ALL_CLEAN_VEHICLES = 'data/20250223_150717/df_all_clean_vehicles.csv'
# FILENAME_DF_ALL_CLEAN_VEHICLES = 'data/20250228_090518/df_all_clean_vehicles.csv'

In [4]:
SEPARATOR_COL = ': '


def col2parts(col: str) -> tuple[str, str]:
    parts = col.split(SEPARATOR_COL)
    if len(parts) == 1:
        return '', parts[0]
    assert len(parts) == 2
    return parts[0], parts[1]


def load_df_all():
    if FILENAME_DF_ALL_CLEAN_VEHICLES is not None:
        print(FILENAME_DF_ALL_CLEAN_VEHICLES)
        df_all = pd.read_csv(FILENAME_DF_ALL_CLEAN_VEHICLES, header=list(range(2)), index_col=[0, 1])
        df_all.index.set_names(['Scenario ID', 'Vehicle'], inplace=True)
        df_all.columns = [SEPARATOR_COL.join(col) for col in df_all.columns]
        
        df_all = df_all[df_all.index.get_level_values('Scenario ID').str.contains('slowness no, forcing no')]  # baseline
        df_all = df_all[df_all.index.get_level_values('Vehicle') != 'V0']  # without human
    
    elif FILENAME_DF_ALL_CLEAN is not None:
        print(FILENAME_DF_ALL_CLEAN)
        df_all = pd.read_csv(FILENAME_DF_ALL_CLEAN, header=list(range(2)), index_col=0)
        df_all.index.set_names(['Scenario ID'], inplace=True)
        df_all.columns = [SEPARATOR_COL.join(col) for col in df_all.columns]
        
        # df_all = df_all[df_all.index.get_level_values('Scenario ID').str.contains('slowness no, forcing no')]  # baseline
        
    else:
        runnames = [
            '20241203_170129_all600',
            '20241213_104400_racing',
            '20241214_122216_racing_passhum',
        ]
        df_all = pd.concat(
            [pd.read_csv(f'data/{runname}/df_all.csv') 
             for runname in runnames],
            keys=runnames,
            names=['runname', 'row']
        )
        assert all(SEPARATOR_COL not in col for col in df_all.columns), df_all.columns
            
    return df_all

df_all = load_df_all()
df_all.info()

data/20250228_090518/df_all_clean.csv
<class 'pandas.core.frame.DataFrame'>
Index: 2889 entries, map-generator/generated-maps/3_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing change of priorities to map-generator/generated-maps/3_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing stops 50% with forcing 50%
Columns: 1638 entries, Violation type: Priority violation to Output of simulation (execution): Near-miss rate
dtypes: bool(4), float64(1610), int64(22), object(2)
memory usage: 36.0+ MB


In [5]:
list(df_all.columns)

['Violation type: Priority violation',
 'Violation type: Speed violation',
 'Violation type: Priority violation and Speed violation',
 'Coordination strategy: Probability of forcing',
 'Coordination strategy: Probability of stops on forcing',
 'Coordination strategy: Rerouting',
 'Static map features: ~ Map ID',
 'Static map features: No. of OPs',
 'Static map features: Connectivity',
 'Static map features: ~ Position',
 'Output of simulation (planning): POD score for MV (V0)',
 'Output of simulation (planning): POD score for AV (V1)',
 'Output of simulation (planning): POD score for AV (V2)',
 'Output of simulation (planning): POD score for AV (V3)',
 'Output of simulation (planning): Mean POD score for AVs (V1-V3)',
 'Output of simulation (planning): POD C 0 for MV (V0)',
 'Output of simulation (planning): POD C 1 for MV (V0)',
 'Output of simulation (planning): POD C 2 for MV (V0)',
 'Output of simulation (planning): POD C 3 for MV (V0)',
 'Output of simulation (planning): POD C 4 f

In [6]:
ID_EXPERIMENT_TEXT = '111'

In [7]:
ID_EXPERIMENT_TEXT = f'{ID_EXPERIMENT_TEXT:03}'
ID_EXPERIMENT_TEXT_FIRST = '100'
ID_EXPERIMENT = tuple(int(c) for c in ID_EXPERIMENT_TEXT) 


def series2values(series):
    dtype = series.dtype
    if dtype == 'bool':
        return series.astype('int').values
    if dtype in ('int64', 'float64'):
        return series.values
    if dtype == 'object':
        return series.astype('category').cat.codes
    raise TypeError(f'{dtype} is not supported')
    

if FILENAME_DF_ALL_CLEAN_VEHICLES is not None or FILENAME_DF_ALL_CLEAN is not None:
    COLUMNS_NONSPLIT = {
        col: series2values
        for col in [
            'Static map features: ~ Position',  # includes i_map, i_position
        ]
    }
    COLUMNS_INPUT = {col: series2values 
                     # for col in (
                     #    'Static map features: Connectivity',
                     # )}
                     for col in df_all.columns 
                     if col2parts(col)[0] != 'Output of simulation (execution)'
                     and not col2parts(col)[1].startswith('~ ')}
    # COLUMNS_INPUT.pop('Static map features: No. of OPs')  # TODO
    # COLUMNS_INPUT.pop('Output of simulation (planning): Mission length for MV (V0)')  # TODO
    # COLUMNS_INPUT.pop('Output of simulation (planning): Mission length for AV (V1)')  # TODO
    # COLUMNS_INPUT.pop('Output of simulation (planning): Mission length for AV (V2)')  # TODO
    # COLUMNS_INPUT.pop('Output of simulation (planning): Mission length for AV (V3)')  # TODO
    # COLUMNS_INPUT.pop('Output of simulation (planning): Total Mission length for AVs (V1-V3)')  # TODO
    # COLUMNS_INPUT.pop('Output of simulation (planning): Total Mission length for AVs (V1-V3)')  # TODO
    # COLUMNS_INPUT = {key: value for key, value in COLUMNS_INPUT.items()
    #                  if 'No. of OPs' not in key and 'POD' not in key}   
    # COLUMNS_INPUT = {key: value for key, value in COLUMNS_INPUT.items()
    #                  if 'POD C' not in key}      
    COLUMNS_INPUT = {
        col: value for col, value in COLUMNS_INPUT.items()
        if ID_EXPERIMENT[0] and re.search(r': POD score$' if FILENAME_DF_ALL_CLEAN_VEHICLES is not None else r'POD score for AV', col)
        or ID_EXPERIMENT[1] and re.search(r': POD C \d+$' if FILENAME_DF_ALL_CLEAN_VEHICLES is not None else r': POD C \d+ for AV', col)
        # or ID_EXPERIMENT[1] and re.search(r': POD D\d \d+$' if FILENAME_DF_ALL_CLEAN_VEHICLES is not None else r': POD D\d for AV', col)
        or ID_EXPERIMENT[2] and re.search(r': Mission length$' if FILENAME_DF_ALL_CLEAN_VEHICLES is not None else r'Mission length for AV', col)
    }
            
    COLUMNS_OUTPUT = {
        col: series2values
        for col in [
            (
                'Output of simulation (execution): No. of completed missions'
                if FILENAME_DF_ALL_CLEAN_VEHICLES is not None else
                'Output of simulation (execution): Total No. of completed missions for AVs (V1-V3)'
            ),
            (
                'Output of simulation (execution): No. of collisions'
                if FILENAME_DF_ALL_CLEAN_VEHICLES is not None else
                'Output of simulation (execution): Collision rate'
            ),
        ]
    }

else:
    COLUMNS_NONSPLIT = {
        'Scenario ID': lambda series: series.values,
    }
    COLUMNS_INPUT = {
        'Coordination strategy': lambda series: series.astype('category').cat.codes.values,
        'isCanPassFirstHum': lambda series: series.astype('int').values,
        'isRacingThroughCrossroadAllowed': lambda series: series.astype('int').values,
        
        'Vehicle ID': lambda series: series.values,
        'Linearization C': lambda series: [
            series.apply(lambda x: -1 if x is None else x[i])
            for i in range(len(series.dropna().iloc[0]))
        ],   
    }
    COLUMNS_OUTPUT = {
        'traveled total, m': lambda series: series.values,
        'No. of completed missions': lambda series: series.values,
        'No. of collisions': lambda series: series.values,
        'No. of near-misses': lambda series: series.values,
    }

COLUMNS_ALL = {**COLUMNS_NONSPLIT, **COLUMNS_INPUT, **COLUMNS_OUTPUT}
COLUMNS_ALL

{'Static map features: ~ Position': <function __main__.series2values(series)>,
 'Output of simulation (planning): POD score for AV (V1)': <function __main__.series2values(series)>,
 'Output of simulation (planning): POD score for AV (V2)': <function __main__.series2values(series)>,
 'Output of simulation (planning): POD score for AV (V3)': <function __main__.series2values(series)>,
 'Output of simulation (planning): Mean POD score for AVs (V1-V3)': <function __main__.series2values(series)>,
 'Output of simulation (planning): POD C 0 for AV (V1)': <function __main__.series2values(series)>,
 'Output of simulation (planning): POD C 1 for AV (V1)': <function __main__.series2values(series)>,
 'Output of simulation (planning): POD C 2 for AV (V1)': <function __main__.series2values(series)>,
 'Output of simulation (planning): POD C 3 for AV (V1)': <function __main__.series2values(series)>,
 'Output of simulation (planning): POD C 4 for AV (V1)': <function __main__.series2values(series)>,
 'Ou

In [8]:
df_inout = df_all[list(COLUMNS_ALL)]
df_inout

,Static map features: ~ Position,Output of simulation (planning): POD score for AV (V1),Output of simulation (planning): POD score for AV (V2),Output of simulation (planning): POD score for AV (V3),Output of simulation (planning): Mean POD score for AVs (V1-V3),Output of simulation (planning): POD C 0 for AV (V1),Output of simulation (planning): POD C 1 for AV (V1),Output of simulation (planning): POD C 2 for AV (V1),Output of simulation (planning): POD C 3 for AV (V1),Output of simulation (planning): POD C 4 for AV (V1),Output of simulation (planning): POD C 5 for AV (V1),Output of simulation (planning): POD C 6 for AV (V1),Output of simulation (planning): POD C 7 for AV (V1),Output of simulation (planning): POD C 8 for AV (V1),Output of simulation (planning): POD C 9 for AV (V1),Output of simulation (planning): POD C 10 for AV (V1),Output of simulation (planning): POD C 11 for AV (V1),Output of simulation (planning): POD C 12 for AV (V1),Output of simulation (planning): POD C 13 for AV (V1),Output of simulation (planning): POD C 14 for AV (V1),Output of simulation (planning): POD C 15 for AV (V1),Output of simulation (planning): POD C 16 for AV (V1),Output of simulation (planning): POD C 17 for AV (V1),Output of simulation (planning): POD C 18 for AV (V1),Output of simulation (planning): POD C 19 for AV (V1),Output of simulation (planning): POD C 20 for AV (V1),Output of simulation (planning): POD C 21 for AV (V1),Output of simulation (planning): POD C 22 for AV (V1),Output of simulation (planning): POD C 23 for AV (V1),Output of simulation (planning): POD C 24 for AV (V1),Output of simulation (planning): POD C 25 for AV (V1),Output of simulation (planning): POD C 26 for AV (V1),Output of simulation (planning): POD C 27 for AV (V1),Output of simulation (planning): POD C 28 for AV (V1),Output of simulation (planning): POD C 29 for AV (V1),Output of simulation (planning): POD C 30 for AV (V1),Output of simulation (planning): POD C 31 for AV (V1),Output of simulation (planning): POD C 32 for AV (V1),Output of simulation (planning): POD C 33 for AV (V1),Output of simulation (planning): POD C 34 for AV (V1),Output of simulation (planning): POD C 35 for AV (V1),Output of simulation (planning): POD C 36 for AV (V1),Output of simulation (planning): POD C 37 for AV (V1),Output of simulation (planning): POD C 38 for AV (V1),Output of simulation (planning): POD C 39 for AV (V1),Output of simulation (planning): POD C 40 for AV (V1),Output of simulation (planning): POD C 41 for AV (V1),Output of simulation (planning): POD C 42 for AV (V1),Output of simulation (planning): POD C 43 for AV (V1),Output of simulation (planning): POD C 44 for AV (V1),...,Output of simulation (planning): POD C 57 for AV (V3),Output of simulation (planning): POD C 58 for AV (V3),Output of simulation (planning): POD C 59 for AV (V3),Output of simulation (planning): POD C 60 for AV (V3),Output of simulation (planning): POD C 61 for AV (V3),Output of simulation (planning): POD C 62 for AV (V3),Output of simulation (planning): POD C 63 for AV (V3),Output of simulation (planning): POD C 64 for AV (V3),Output of simulation (planning): POD C 65 for AV (V3),Output of simulation (planning): POD C 66 for AV (V3),Output of simulation (planning): POD C 67 for AV (V3),Output of simulation (planning): POD C 68 for AV (V3),Output of simulation (planning): POD C 69 for AV (V3),Output of simulation (planning): POD C 70 for AV (V3),Output of simulation (planning): POD C 71 for AV (V3),Output of simulation (planning): POD C 72 for AV (V3),Output of simulation (planning): POD C 73 for AV (V3),Output of simulation (planning): POD C 74 for AV (V3),Output of simulation (planning): POD C 75 for AV (V3),Output of simulation (planning): POD C 76 for AV (V3),Output of simulation (planning): POD C 77 for AV (V3),Output of simulation (planning): POD C 78 for AV (V3),Output of simulation (planning): POD C 79 for AV (V3),Output of simulation (planning): POD C 80 for AV (V3),Output of simulati

In [9]:
def parse_tuple_string(tuple_string):
    if pd.isna(tuple_string):
        return None
    return ast.literal_eval(tuple_string)


def preprocess_inout(df_inout):
    df_inout = df_inout.copy()
    
    linearization_columns = [col for col in df_inout.columns if isinstance(col, str) and col.startswith('Linearization')]
    for col in linearization_columns:
        df_inout[col] = df_inout[col].apply(parse_tuple_string)
        
    dict_preprocessed = {}
    for col, series2data in COLUMNS_ALL.items():
        data = series2data(df_inout[col])
        if not isinstance(data, list):
            dict_preprocessed[col] = data
        else:
            for i, series in enumerate(data):
                dict_preprocessed[f'{col}#{i}'] = series
    
    return pd.DataFrame(dict_preprocessed)
    

df_preprocessed = preprocess_inout(df_inout)
df_preprocessed

,Static map features: ~ Position,Output of simulation (planning): POD score for AV (V1),Output of simulation (planning): POD score for AV (V2),Output of simulation (planning): POD score for AV (V3),Output of simulation (planning): Mean POD score for AVs (V1-V3),Output of simulation (planning): POD C 0 for AV (V1),Output of simulation (planning): POD C 1 for AV (V1),Output of simulation (planning): POD C 2 for AV (V1),Output of simulation (planning): POD C 3 for AV (V1),Output of simulation (planning): POD C 4 for AV (V1),Output of simulation (planning): POD C 5 for AV (V1),Output of simulation (planning): POD C 6 for AV (V1),Output of simulation (planning): POD C 7 for AV (V1),Output of simulation (planning): POD C 8 for AV (V1),Output of simulation (planning): POD C 9 for AV (V1),Output of simulation (planning): POD C 10 for AV (V1),Output of simulation (planning): POD C 11 for AV (V1),Output of simulation (planning): POD C 12 for AV (V1),Output of simulation (planning): POD C 13 for AV (V1),Output of simulation (planning): POD C 14 for AV (V1),Output of simulation (planning): POD C 15 for AV (V1),Output of simulation (planning): POD C 16 for AV (V1),Output of simulation (planning): POD C 17 for AV (V1),Output of simulation (planning): POD C 18 for AV (V1),Output of simulation (planning): POD C 19 for AV (V1),Output of simulation (planning): POD C 20 for AV (V1),Output of simulation (planning): POD C 21 for AV (V1),Output of simulation (planning): POD C 22 for AV (V1),Output of simulation (planning): POD C 23 for AV (V1),Output of simulation (planning): POD C 24 for AV (V1),Output of simulation (planning): POD C 25 for AV (V1),Output of simulation (planning): POD C 26 for AV (V1),Output of simulation (planning): POD C 27 for AV (V1),Output of simulation (planning): POD C 28 for AV (V1),Output of simulation (planning): POD C 29 for AV (V1),Output of simulation (planning): POD C 30 for AV (V1),Output of simulation (planning): POD C 31 for AV (V1),Output of simulation (planning): POD C 32 for AV (V1),Output of simulation (planning): POD C 33 for AV (V1),Output of simulation (planning): POD C 34 for AV (V1),Output of simulation (planning): POD C 35 for AV (V1),Output of simulation (planning): POD C 36 for AV (V1),Output of simulation (planning): POD C 37 for AV (V1),Output of simulation (planning): POD C 38 for AV (V1),Output of simulation (planning): POD C 39 for AV (V1),Output of simulation (planning): POD C 40 for AV (V1),Output of simulation (planning): POD C 41 for AV (V1),Output of simulation (planning): POD C 42 for AV (V1),Output of simulation (planning): POD C 43 for AV (V1),Output of simulation (planning): POD C 44 for AV (V1),...,Output of simulation (planning): POD C 57 for AV (V3),Output of simulation (planning): POD C 58 for AV (V3),Output of simulation (planning): POD C 59 for AV (V3),Output of simulation (planning): POD C 60 for AV (V3),Output of simulation (planning): POD C 61 for AV (V3),Output of simulation (planning): POD C 62 for AV (V3),Output of simulation (planning): POD C 63 for AV (V3),Output of simulation (planning): POD C 64 for AV (V3),Output of simulation (planning): POD C 65 for AV (V3),Output of simulation (planning): POD C 66 for AV (V3),Output of simulation (planning): POD C 67 for AV (V3),Output of simulation (planning): POD C 68 for AV (V3),Output of simulation (planning): POD C 69 for AV (V3),Output of simulation (planning): POD C 70 for AV (V3),Output of simulation (planning): POD C 71 for AV (V3),Output of simulation (planning): POD C 72 for AV (V3),Output of simulation (planning): POD C 73 for AV (V3),Output of simulation (planning): POD C 74 for AV (V3),Output of simulation (planning): POD C 75 for AV (V3),Output of simulation (planning): POD C 76 for AV (V3),Output of simulation (planning): POD C 77 for AV (V3),Output of simulation (planning): POD C 78 for AV (V3),Output of simulation (planning): POD C 79 for AV (V3),Output of simulation (planning): POD C 80 for AV (V3),Output of simulati

In [10]:
def show(obj, title=None):
    if title is not None:
        display(HTML(f"<h3>{title}</h3>"))
    display(obj)


def shuffle_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    # Shuffle by Scenario ID
    unique_scenarios = df['Scenario ID'].unique()  # Get unique Scenario IDs
    shuffled_scenarios = pd.Series(unique_scenarios).sample(frac=1, random_state=1).tolist()  # Shuffle Scenario IDs
    
    # Reorder the dataframe by the shuffled Scenario IDs
    df_shuffled = pd.concat([df[df['Scenario ID'] == scenario] for scenario in shuffled_scenarios])
    
    # Reset index (optional)
    df_shuffled = df_shuffled.reset_index(drop=True)
    
    return df_shuffled
    """
    return df.sample(frac=1, random_state=1)


def split_train_test(df):
    test_size = 0.2
    
    if not COLUMNS_NONSPLIT:
        return sklearn.model_selection.train_test_split(df, test_size=test_size, random_state=1)
    
    gss = sklearn.model_selection.GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=1)
    column, = list(COLUMNS_NONSPLIT)
    groups = df[column]
    
    # Split the data
    for train_idx, test_idx in gss.split(df, groups=groups):
        return shuffle_df(df.iloc[train_idx]), shuffle_df(df.iloc[test_idx])


df_train, df_test = split_train_test(df_preprocessed)
show(df_train, 'df_train')
show(df_test, 'df_test')
if 'Static map features: ~ Position' in df_train.columns:
    assert not set(df_train['Static map features: ~ Position']) & set(df_test['Static map features: ~ Position']) 

,Static map features: ~ Position,Output of simulation (planning): POD score for AV (V1),Output of simulation (planning): POD score for AV (V2),Output of simulation (planning): POD score for AV (V3),Output of simulation (planning): Mean POD score for AVs (V1-V3),Output of simulation (planning): POD C 0 for AV (V1),Output of simulation (planning): POD C 1 for AV (V1),Output of simulation (planning): POD C 2 for AV (V1),Output of simulation (planning): POD C 3 for AV (V1),Output of simulation (planning): POD C 4 for AV (V1),Output of simulation (planning): POD C 5 for AV (V1),Output of simulation (planning): POD C 6 for AV (V1),Output of simulation (planning): POD C 7 for AV (V1),Output of simulation (planning): POD C 8 for AV (V1),Output of simulation (planning): POD C 9 for AV (V1),Output of simulation (planning): POD C 10 for AV (V1),Output of simulation (planning): POD C 11 for AV (V1),Output of simulation (planning): POD C 12 for AV (V1),Output of simulation (planning): POD C 13 for AV (V1),Output of simulation (planning): POD C 14 for AV (V1),Output of simulation (planning): POD C 15 for AV (V1),Output of simulation (planning): POD C 16 for AV (V1),Output of simulation (planning): POD C 17 for AV (V1),Output of simulation (planning): POD C 18 for AV (V1),Output of simulation (planning): POD C 19 for AV (V1),Output of simulation (planning): POD C 20 for AV (V1),Output of simulation (planning): POD C 21 for AV (V1),Output of simulation (planning): POD C 22 for AV (V1),Output of simulation (planning): POD C 23 for AV (V1),Output of simulation (planning): POD C 24 for AV (V1),Output of simulation (planning): POD C 25 for AV (V1),Output of simulation (planning): POD C 26 for AV (V1),Output of simulation (planning): POD C 27 for AV (V1),Output of simulation (planning): POD C 28 for AV (V1),Output of simulation (planning): POD C 29 for AV (V1),Output of simulation (planning): POD C 30 for AV (V1),Output of simulation (planning): POD C 31 for AV (V1),Output of simulation (planning): POD C 32 for AV (V1),Output of simulation (planning): POD C 33 for AV (V1),Output of simulation (planning): POD C 34 for AV (V1),Output of simulation (planning): POD C 35 for AV (V1),Output of simulation (planning): POD C 36 for AV (V1),Output of simulation (planning): POD C 37 for AV (V1),Output of simulation (planning): POD C 38 for AV (V1),Output of simulation (planning): POD C 39 for AV (V1),Output of simulation (planning): POD C 40 for AV (V1),Output of simulation (planning): POD C 41 for AV (V1),Output of simulation (planning): POD C 42 for AV (V1),Output of simulation (planning): POD C 43 for AV (V1),Output of simulation (planning): POD C 44 for AV (V1),...,Output of simulation (planning): POD C 57 for AV (V3),Output of simulation (planning): POD C 58 for AV (V3),Output of simulation (planning): POD C 59 for AV (V3),Output of simulation (planning): POD C 60 for AV (V3),Output of simulation (planning): POD C 61 for AV (V3),Output of simulation (planning): POD C 62 for AV (V3),Output of simulation (planning): POD C 63 for AV (V3),Output of simulation (planning): POD C 64 for AV (V3),Output of simulation (planning): POD C 65 for AV (V3),Output of simulation (planning): POD C 66 for AV (V3),Output of simulation (planning): POD C 67 for AV (V3),Output of simulation (planning): POD C 68 for AV (V3),Output of simulation (planning): POD C 69 for AV (V3),Output of simulation (planning): POD C 70 for AV (V3),Output of simulation (planning): POD C 71 for AV (V3),Output of simulation (planning): POD C 72 for AV (V3),Output of simulation (planning): POD C 73 for AV (V3),Output of simulation (planning): POD C 74 for AV (V3),Output of simulation (planning): POD C 75 for AV (V3),Output of simulation (planning): POD C 76 for AV (V3),Output of simulation (planning): POD C 77 for AV (V3),Output of simulation (planning): POD C 78 for AV (V3),Output of simulation (planning): POD C 79 for AV (V3),Output of simulation (planning): POD C 80 for AV (V3),Output of simulati

,Static map features: ~ Position,Output of simulation (planning): POD score for AV (V1),Output of simulation (planning): POD score for AV (V2),Output of simulation (planning): POD score for AV (V3),Output of simulation (planning): Mean POD score for AVs (V1-V3),Output of simulation (planning): POD C 0 for AV (V1),Output of simulation (planning): POD C 1 for AV (V1),Output of simulation (planning): POD C 2 for AV (V1),Output of simulation (planning): POD C 3 for AV (V1),Output of simulation (planning): POD C 4 for AV (V1),Output of simulation (planning): POD C 5 for AV (V1),Output of simulation (planning): POD C 6 for AV (V1),Output of simulation (planning): POD C 7 for AV (V1),Output of simulation (planning): POD C 8 for AV (V1),Output of simulation (planning): POD C 9 for AV (V1),Output of simulation (planning): POD C 10 for AV (V1),Output of simulation (planning): POD C 11 for AV (V1),Output of simulation (planning): POD C 12 for AV (V1),Output of simulation (planning): POD C 13 for AV (V1),Output of simulation (planning): POD C 14 for AV (V1),Output of simulation (planning): POD C 15 for AV (V1),Output of simulation (planning): POD C 16 for AV (V1),Output of simulation (planning): POD C 17 for AV (V1),Output of simulation (planning): POD C 18 for AV (V1),Output of simulation (planning): POD C 19 for AV (V1),Output of simulation (planning): POD C 20 for AV (V1),Output of simulation (planning): POD C 21 for AV (V1),Output of simulation (planning): POD C 22 for AV (V1),Output of simulation (planning): POD C 23 for AV (V1),Output of simulation (planning): POD C 24 for AV (V1),Output of simulation (planning): POD C 25 for AV (V1),Output of simulation (planning): POD C 26 for AV (V1),Output of simulation (planning): POD C 27 for AV (V1),Output of simulation (planning): POD C 28 for AV (V1),Output of simulation (planning): POD C 29 for AV (V1),Output of simulation (planning): POD C 30 for AV (V1),Output of simulation (planning): POD C 31 for AV (V1),Output of simulation (planning): POD C 32 for AV (V1),Output of simulation (planning): POD C 33 for AV (V1),Output of simulation (planning): POD C 34 for AV (V1),Output of simulation (planning): POD C 35 for AV (V1),Output of simulation (planning): POD C 36 for AV (V1),Output of simulation (planning): POD C 37 for AV (V1),Output of simulation (planning): POD C 38 for AV (V1),Output of simulation (planning): POD C 39 for AV (V1),Output of simulation (planning): POD C 40 for AV (V1),Output of simulation (planning): POD C 41 for AV (V1),Output of simulation (planning): POD C 42 for AV (V1),Output of simulation (planning): POD C 43 for AV (V1),Output of simulation (planning): POD C 44 for AV (V1),...,Output of simulation (planning): POD C 57 for AV (V3),Output of simulation (planning): POD C 58 for AV (V3),Output of simulation (planning): POD C 59 for AV (V3),Output of simulation (planning): POD C 60 for AV (V3),Output of simulation (planning): POD C 61 for AV (V3),Output of simulation (planning): POD C 62 for AV (V3),Output of simulation (planning): POD C 63 for AV (V3),Output of simulation (planning): POD C 64 for AV (V3),Output of simulation (planning): POD C 65 for AV (V3),Output of simulation (planning): POD C 66 for AV (V3),Output of simulation (planning): POD C 67 for AV (V3),Output of simulation (planning): POD C 68 for AV (V3),Output of simulation (planning): POD C 69 for AV (V3),Output of simulation (planning): POD C 70 for AV (V3),Output of simulation (planning): POD C 71 for AV (V3),Output of simulation (planning): POD C 72 for AV (V3),Output of simulation (planning): POD C 73 for AV (V3),Output of simulation (planning): POD C 74 for AV (V3),Output of simulation (planning): POD C 75 for AV (V3),Output of simulation (planning): POD C 76 for AV (V3),Output of simulation (planning): POD C 77 for AV (V3),Output of simulation (planning): POD C 78 for AV (V3),Output of simulation (planning): POD C 79 for AV (V3),Output of simulation (planning): POD C 80 for AV (V3),Output of simulati

In [11]:
df_train.sort_values(by='Scenario ID')

,Static map features: ~ Position,Output of simulation (planning): POD score for AV (V1),Output of simulation (planning): POD score for AV (V2),Output of simulation (planning): POD score for AV (V3),Output of simulation (planning): Mean POD score for AVs (V1-V3),Output of simulation (planning): POD C 0 for AV (V1),Output of simulation (planning): POD C 1 for AV (V1),Output of simulation (planning): POD C 2 for AV (V1),Output of simulation (planning): POD C 3 for AV (V1),Output of simulation (planning): POD C 4 for AV (V1),Output of simulation (planning): POD C 5 for AV (V1),Output of simulation (planning): POD C 6 for AV (V1),Output of simulation (planning): POD C 7 for AV (V1),Output of simulation (planning): POD C 8 for AV (V1),Output of simulation (planning): POD C 9 for AV (V1),Output of simulation (planning): POD C 10 for AV (V1),Output of simulation (planning): POD C 11 for AV (V1),Output of simulation (planning): POD C 12 for AV (V1),Output of simulation (planning): POD C 13 for AV (V1),Output of simulation (planning): POD C 14 for AV (V1),Output of simulation (planning): POD C 15 for AV (V1),Output of simulation (planning): POD C 16 for AV (V1),Output of simulation (planning): POD C 17 for AV (V1),Output of simulation (planning): POD C 18 for AV (V1),Output of simulation (planning): POD C 19 for AV (V1),Output of simulation (planning): POD C 20 for AV (V1),Output of simulation (planning): POD C 21 for AV (V1),Output of simulation (planning): POD C 22 for AV (V1),Output of simulation (planning): POD C 23 for AV (V1),Output of simulation (planning): POD C 24 for AV (V1),Output of simulation (planning): POD C 25 for AV (V1),Output of simulation (planning): POD C 26 for AV (V1),Output of simulation (planning): POD C 27 for AV (V1),Output of simulation (planning): POD C 28 for AV (V1),Output of simulation (planning): POD C 29 for AV (V1),Output of simulation (planning): POD C 30 for AV (V1),Output of simulation (planning): POD C 31 for AV (V1),Output of simulation (planning): POD C 32 for AV (V1),Output of simulation (planning): POD C 33 for AV (V1),Output of simulation (planning): POD C 34 for AV (V1),Output of simulation (planning): POD C 35 for AV (V1),Output of simulation (planning): POD C 36 for AV (V1),Output of simulation (planning): POD C 37 for AV (V1),Output of simulation (planning): POD C 38 for AV (V1),Output of simulation (planning): POD C 39 for AV (V1),Output of simulation (planning): POD C 40 for AV (V1),Output of simulation (planning): POD C 41 for AV (V1),Output of simulation (planning): POD C 42 for AV (V1),Output of simulation (planning): POD C 43 for AV (V1),Output of simulation (planning): POD C 44 for AV (V1),...,Output of simulation (planning): POD C 57 for AV (V3),Output of simulation (planning): POD C 58 for AV (V3),Output of simulation (planning): POD C 59 for AV (V3),Output of simulation (planning): POD C 60 for AV (V3),Output of simulation (planning): POD C 61 for AV (V3),Output of simulation (planning): POD C 62 for AV (V3),Output of simulation (planning): POD C 63 for AV (V3),Output of simulation (planning): POD C 64 for AV (V3),Output of simulation (planning): POD C 65 for AV (V3),Output of simulation (planning): POD C 66 for AV (V3),Output of simulation (planning): POD C 67 for AV (V3),Output of simulation (planning): POD C 68 for AV (V3),Output of simulation (planning): POD C 69 for AV (V3),Output of simulation (planning): POD C 70 for AV (V3),Output of simulation (planning): POD C 71 for AV (V3),Output of simulation (planning): POD C 72 for AV (V3),Output of simulation (planning): POD C 73 for AV (V3),Output of simulation (planning): POD C 74 for AV (V3),Output of simulation (planning): POD C 75 for AV (V3),Output of simulation (planning): POD C 76 for AV (V3),Output of simulation (planning): POD C 77 for AV (V3),Output of simulation (planning): POD C 78 for AV (V3),Output of simulation (planning): POD C 79 for AV (V3),Output of simulation (planning): POD C 80 for AV (V3),Output of simulati

# Logistic regression (as a baseline)

In [12]:
def split_df_to_X_y(df):
    columns_input_df = [col for col in df.columns
                        if (col if not isinstance(col, str) else col.split('#')[0]) in COLUMNS_INPUT]
    columns_output_df = list(COLUMNS_OUTPUT)
    assert set(COLUMNS_NONSPLIT) | set(columns_input_df) | set(columns_output_df) == set(df.columns)
    
    X = df[columns_input_df]
    y = df[columns_output_df]    
    return X, y


def run_regression(df_train, df_test):
    X_train, y_train = split_df_to_X_y(df_train)
    X_test, y_test = split_df_to_X_y(df_test)
    
    model = sklearn.linear_model.LinearRegression()
    model.fit(X_train, y_train)
    ndarray_predictions = model.predict(X_test)
    df_predictions = pd.DataFrame(ndarray_predictions, columns=y_test.columns)
    return df_predictions


df_predictions_regression = run_regression(df_train, df_test)
df_predictions_regression

,Output of simulation (execution): Total No. of completed missions for AVs (V1-V3),Output of simulation (execution): Collision rate
0,29.274107,0.311680
1,14.292325,0.155233
2,36.467251,0.488640
3,14.292325,0.155233
4,32.283173,0.206146
...,...,...
573,18.059983,0.501797
574,11.121350,0.701479
575,15.533027,0.676583
576,19.953248,2.069058


In [13]:
def save_and_show(fig, basename):  # to avoid inlining large image data into the notebook file
    filename = f'{DIRECTORY_DATA}/{basename}-{random.random()}.png'
    fig.savefig(filename)
    
    # The `random` is because of https://stackoverflow.com/a/43640705.
    display(HTML(f'<img src="{filename}?{random.random()}" alt="{basename}" title="{datetime.datetime.now()}" />'))
    
    plt.close(fig)
    
    return filename


def evaluate_and_plot_column(df_test, df_predictions, column, *, is_plot=True):
    y_test_column = df_test[column]
    predictions_column = df_predictions[column]
    
    r2 = sklearn.metrics.r2_score(y_test_column, predictions_column)
    name = col2parts(column)[1]
    print(f"{name}:")
    print(f"- R^2 Score: {r2}")
    
    if not is_plot:
        return

    # Plot results for each output column
    fig = plt.figure(figsize=(10, 6))
    plt.scatter(y_test_column, predictions_column, color='blue', alpha=0.5)
    plt.plot([y_test_column.min(), y_test_column.max()], [y_test_column.min(), y_test_column.max()], 'k--', lw=2)
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'Actual vs Predicted Values for {name}')
    plt.grid(True)
    save_and_show(fig, f'Actual_vs_Predicted_Values_{name}')


def evaluate_and_plot_all_columns(df_test, df_predictions):
    for column in COLUMNS_OUTPUT:
        evaluate_and_plot_column(df_test, df_predictions, column, is_plot=False)
        
        
evaluate_and_plot_all_columns(df_test, df_predictions_regression)

Total No. of completed missions for AVs (V1-V3):
- R^2 Score: -0.007351332098973318
Collision rate:
- R^2 Score: -0.31869932034109816


# AutoGluon

In [14]:
def run_autogluon(df_train, df_test):
    X_train, y_train = split_df_to_X_y(df_train)
    X_test, y_test = split_df_to_X_y(df_test)
    
    # Train AutoGluon models
    predictors = []
    df_predictions = pd.DataFrame()
    for i_column, column in enumerate(COLUMNS_OUTPUT):
        print(f'{column=}:')
        df_train_predictor = pd.concat([X_train, y_train[[column]]], axis=1)
        preset = 'medium'  # medium (~1 min.), good (~15 min.), high (~2 h)
        
        predictor = autogluon.tabular.TabularPredictor(
            label=column, 
            eval_metric='r2', 
            problem_type='regression',
        ).fit(
            df_train_predictor,
            presets=preset,
            hyperparameters={
                'GBM': {},       # LightGBM (TODO: something like `GBMLarge`)
                'XGB': {},       # XGBoost
                'RF': {},        # Random Forest
                'XT': {},        # Extra Trees
                # 'CAT': {},      # CatBoost, omitted if slow
                # 'NN': {},       # Neural net, if you want it
                # 'LR': {},       # Linear model
                # 'KNN': {},      # K-Nearest Neighbors
            },
        )
        predictors.append(predictor)
        
        df_predictions[column] = predictor.predict(X_test)

        # Leaderboard - Display a table of different models and their performance
        df_test_predictor = pd.concat([X_test, y_test[[column]]], axis=1)
        leaderboard = predictor.leaderboard(df_test_predictor, silent=True)
        rundir = os.path.dirname(FILENAME_DF_ALL_CLEAN)
        _leaderboard, r2 = dynmodel.process_leaderboard(leaderboard, rundir, column, f'{preset} {ID_EXPERIMENT_TEXT}')        
        
        filename_results = f'{rundir}/{preset} results.csv' 
        with open(
                filename_results, 
                'w' if ID_EXPERIMENT_TEXT == ID_EXPERIMENT_TEXT_FIRST and i_column == 0 else 'a'
        ) as file:
            if i_column == 0:
                file.write(ID_EXPERIMENT_TEXT)
            file.write(f',{r2:.3f}')
            if i_column == len(COLUMNS_OUTPUT) - 1:
                file.write('\n')
        
        # Feature importance on training data
        # show(
        #     predictor.feature_importance(df_train_predictor),
        #     'feature_importance(df_train_predictor)'
        # )
        # 
        # # Feature importance on test data
        # show(
        #     predictor.feature_importance(df_test_predictor),
        #     'feature_importance(df_test_predictor)'
        # )
        # Example: SHAP values for a specific model
        # shap_values = predictor.get_model_shap_values(df_test_predictor, model='LightGBM')
        # show(shap_values, 'shap_values')  # SHAP values for each feature and each prediction
        
    return predictors, df_predictions


predictors, df_predictions_autogluon = run_autogluon(df_train, df_test)
df_predictions_autogluon

No path specified. Models will be saved in: "AutogluonModels/ag-20250302_095819"
Preset alias specified: 'medium' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.2
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #141~20.04.1-Ubuntu SMP Thu Jan 16 18:38:51 UTC 2025
CPU Count:          16
Memory Avail:       1.99 GB / 31.09 GB (6.4%)
Disk Space Avail:   173.93 GB / 693.60 GB (25.1%)
Presets specified: ['medium']
Beginning AutoGluon training ...
AutoGluon will save models to "/home/olga/coordination_oru/scenario-analysis/AutogluonModels/ag-20250302_095819"
Train Data Rows:    2311
Train Data Columns: 309
Label Column:       Output of simulation (execution): Total No. of completed missions for AVs (V1-V3)
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Avai

column='Output of simulation (execution): Total No. of completed missions for AVs (V1-V3)':


	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 2 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Useless Original Features (Count: 5): ['Output of simulation (planning): POD C 0 for AV (V3)', 'Output of simulation (planning): POD C 1 for AV (V3)', 'Output of simulation (planning): POD C 2 for AV (V3)', 'Output of simulation (planning): POD C 3 for AV (V3)', 'Output of simulation (planning): POD C 4 for AV (V3)']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all rows.
		These features do not need to be present at inference time.
	Unused Original Features (Count: 27): ['Output of simulation (planning): POD C 9 f

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
2,LightGBM,0.497,0.766,r2,0.003,0.002,4.883,0.003,0.002,4.883,1,True,1
3,WeightedEnsemble_L2,0.497,0.766,r2,0.005,0.002,4.932,0.001,0.001,0.050,2,True,5
4,XGBoost,0.490,0.763,r2,0.013,0.004,1.447,0.013,0.004,1.447,1,True,4
0,RandomForest,0.539,0.758,r2,0.057,0.035,1.781,0.057,0.035,1.781,1,True,2
1,ExtraTrees,0.527,0.758,r2,0.048,0.035,0.915,0.048,0.035,0.915,1,True,3


No path specified. Models will be saved in: "AutogluonModels/ag-20250302_095833"
Preset alias specified: 'medium' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.2
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #141~20.04.1-Ubuntu SMP Thu Jan 16 18:38:51 UTC 2025
CPU Count:          16
Memory Avail:       1.82 GB / 31.09 GB (5.9%)
Disk Space Avail:   173.91 GB / 693.60 GB (25.1%)
Presets specified: ['medium']
Beginning AutoGluon training ...
AutoGluon will save models to "/home/olga/coordination_oru/scenario-analysis/AutogluonModels/ag-20250302_095833"
Train Data Rows:    2311
Train Data Columns: 309
Label Column:       Output of simulation (execution): Collision rate
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    

column='Output of simulation (execution): Collision rate':


	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Useless Original Features (Count: 5): ['Output of simulation (planning): POD C 0 for AV (V3)', 'Output of simulation (planning): POD C 1 for AV (V3)', 'Output of simulation (planning): POD C 2 for AV (V3)', 'Output of simulation (planning): POD C 3 for AV (V3)', 'Output of simulation (planning): POD C 4 for AV (V3)']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all rows.
		These features do not need to be present at inference time.
	Unused Original Features (Count: 27): ['Output of simulation (planning): POD C 9 for AV (V1)', 'Output of simulation (planning): POD C 15 for AV (V1)', 'Output of simulation (planning): POD C 19 for AV (V1)', 'Output of simulation (planning): POD C 26 for AV (V1)', 'Output of simulation (planning): POD C 7 for AV (V2)', 'Output of simulation (planning): POD C 16 for AV (V2)', 'Output of simulation (

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
2,XGBoost,0.044,0.050,r2,0.012,0.004,3.061,0.012,0.004,3.061,1,True,4
3,WeightedEnsemble_L2,0.044,0.050,r2,0.013,0.005,3.112,0.002,0.001,0.051,2,True,5
4,LightGBM,0.036,0.047,r2,0.002,0.001,1.037,0.002,0.001,1.037,1,True,1
0,ExtraTrees,0.052,0.017,r2,0.057,0.035,1.030,0.057,0.035,1.030,1,True,3
1,RandomForest,0.051,0.017,r2,0.056,0.040,2.068,0.056,0.040,2.068,1,True,2


,Output of simulation (execution): Total No. of completed missions for AVs (V1-V3),Output of simulation (execution): Collision rate
Scenario ID,,
"map-generator/generated-maps/3_with_bridges/scenario6-5.json, passhum 0, slowness with rerouting, forcing no",28.579018,0.277992
"map-generator/generated-maps/3_without_bridges/scenario3-9.json, passhum 0, slowness without rerouting, forcing stops",20.551651,0.350097
"map-generator/generated-maps/3_without_bridges/scenario10-7.json, passhum 0, slowness no, forcing change of priorities",21.782949,0.397837
"map-generator/generated-maps/3_without_bridges/scenario3-9.json, passhum 0, slowness no, forcing change of priorities",20.551651,0.350097
"map-generator/generated-maps/3_without_bridges/scenario6-5.json, passhum 0, slowness without rerouting, forcing stops 50%",31.475662,0.299337
...,...,...
"map-generator/generated-maps/3_with_bridges/scenario4-4.json, passhum 0, slowness without rerouting, forcing change of priorities",17.902971,0.382025
"map-generator/generated-maps/3_with_bridges/scenario5-10.json, passhum 0, slowness with rerouting, forcing stops 50% with forcing 50%",12.744311,0.494433
"map-generator/generated-maps/3_with_bridges/scenario3-3.json, passhum 0, slowness no, forcing stops 50% with forcing 50%",19.102615,0.484020


## evaluate_and_plot_all_columns

In [15]:
import collections

def print_stats():
    for col in [
        'Output of simulation (execution): No. of collisions',
    ]:
        print(col)
        for df in [
            df_train,
            df_test,
            df_predictions_autogluon,
        ]:
            print(collections.Counter(df[col]))
        
        
print_stats()

Output of simulation (execution): No. of collisions


KeyError: 'Output of simulation (execution): No. of collisions'

In [ ]:
evaluate_and_plot_all_columns(df_test, df_predictions_autogluon)

## explain_predictions

In [ ]:
def explain_predictions(predictors):
    # X_test, y_test = split_df_to_X_y(df_test)
    
    for column, predictor in zip(COLUMNS_OUTPUT, predictors):
        # df_test_predictor = pd.concat([X_test, y_test[[column]]], axis=1)
        if column not in (
            'Output of simulation (execution): Total No. of completed missions for AVs (V1-V3)',
            'Output of simulation (execution): Collision rate',
        ):
            continue
        
        for model in 'LightGBM', 'XGBoost':
            tree_model = predictor._trainer.load_model(model)
            if model == 'LightGBM':
                tree_importance = tree_model.model.feature_importance(importance_type='gain')
            elif model == 'XGBoost':
                tree_importance = tree_model.model.feature_importances_
            else:
                raise ValueError(model)
            # show(tree_importance, column)
                
            
            feature_names = predictor.feature_metadata.get_features()
            df = pd.DataFrame.from_dict(
                {name: {'importance': value} for name, value in zip(feature_names, tree_importance)},
                orient='index'
            )
            # print(df.index)
            # Group indexes by their base name before `#`
            df['group'] = df.index.str.extract(r'^(.+?)(?:#\d+)?$', expand=False)
            df = df.groupby('group')['importance'].sum().to_frame()
            
            df.sort_values(by='importance', ascending=False, inplace=True)
            show(df, f'{model}: {column}')
        
        
explain_predictions(predictors)